## Use pretrained network to output keypoint's description

In [1]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment
import sys

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
from descriptor_CNN3 import DesNet
model = DesNet()
model.eval()
if args.cuda:
    model.cuda()
weight_path = "checkpoint.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])


In [3]:
# load patches
# query
patches_q_dir = "../keypoint_detector/patches_query.pt"
patches_q = torch.load(patches_q_dir)
print("query patches: ", patches_q.shape)
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

# dataset
patches_d_dir = "../keypoint_detector/patches_images.pt"
patches_d = torch.load(patches_d_dir)
print("dataset patches: ", patches_d.shape)
patches_d =  patches_d.view(-1, 1, 32, 32).cuda()

query patches:  torch.Size([35, 30, 1, 32, 32])
dataset patches:  torch.Size([140, 30, 1, 32, 32])


In [4]:
# query
description_q = model(patches_q)
description_q = description_q.view(-1, 30, 128)

# dataset
description_d = model(patches_d)
description_d = description_d.view(-1, 30, 128)

In [5]:
print("query description: ", description_q.shape)
print("dataset description: ", description_d.shape) 

query description:  torch.Size([35, 30, 128])
dataset description:  torch.Size([140, 30, 128])


In [6]:
torch.save(description_q, "query_description.pt")
torch.save(description_d, "data_description.pt") 

In [7]:
c = torch.cat((description_q, description_d), 0)
torch.save(c, "feature_descriptors.pt")
print("feature_descriptors: ", c.shape)

feature_descriptors:  torch.Size([175, 30, 128])
